In [1]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("state_of_the_union.txt")
documents = loader.load()

In [4]:
# documents

In [6]:
OPENAI_API_KEY = ""

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
db = FAISS.from_documents(texts, embeddings)
retriever = db.as_retriever()

In [14]:
from langchain.tools.retriever import create_retriever_tool
tool = create_retriever_tool(
    retriever,
    "search_state_of_union",
    "Searches and returns excerpts from the 2022 State of the Union."
)
tools= [tool]

In [9]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [15]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [16]:
result = agent_executor.invoke({"input": "hi, im bob"})

In [17]:
result

{'input': 'hi, im bob', 'output': 'Hello Bob! How can I assist you today?'}

In [18]:
result = agent_executor.invoke(
    {
        "input": "what did the president say about ketanji brown jackson in the most recent state of the union?"
    }
)

In [19]:
result

{'input': 'what did the president say about ketanji brown jackson in the most recent state of the union?',
 'output': "In the most recent State of the Union address, President Biden mentioned Judge Ketanji Brown Jackson as his nominee for the United States Supreme Court. He described her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. The President highlighted her background as a former top litigator in private practice, a former federal public defender, and being from a family of public school educators and police officers. He also mentioned the broad range of support she has received since her nomination.\n\nIf you would like more information or specific details from the State of the Union address regarding Judge Ketanji Brown Jackson, feel free to ask!"}